<a href="https://colab.research.google.com/github/janiosl/python.ds/blob/master/ml/PyTorch/ML_nnMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MLP SEM SINTONIZAÇÃO DE HIPERPARÂMETROS

In [31]:
#Bibliotecas para carga, tratamento e visualização
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

#Importações de PyTorch para criação de modelos e preparação de variáveis
import torch
from torch.autograd import Variable


#Biblioteca para cálculo das métricas de avaliação e separação dos datasets
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

#Importações do
import torch.nn as nn
import torch.nn.functional as F

In [32]:
#Definção dos títulos das colunas
headers = ['ESCT', 'NDEP', 'RENDA', 'TIPOR', 'VBEM', 'NPARC',
           'VPARC', 'TEL', 'IDADE', 'RESMS', 'ENTRADA', 'CLASSE']


#Carga do conjunto de treino
arquivo = 'https://raw.githubusercontent.com/MLRG-CEFET-RJ/ml-class/master/ml-t3/datasets/credtrain.txt'
data_train = pd.read_csv(arquivo, sep='\t', header=None, names=headers)


#Transoformação dos atributos e da classe alvo em matrizes
X_train_ = np.array(data_train.iloc[:, 0:11])
y_train_ = np.array(data_train['CLASSE'])


#Separação de treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_train_, y_train_,
                                                  train_size=0.2,
                                                  random_state=31)


#Carga do conjunto de teste
arquivo = 'https://raw.githubusercontent.com/MLRG-CEFET-RJ/ml-class/master/ml-t3/datasets/credtest.txt'
data_test = pd.read_csv(arquivo, sep='\t', header=None, names=headers)


#Transformação dos atributos e da classe alvo em matrizes
X_test = np.array(data_test.iloc[:, 0:11])
y_test = np.array(data_test['CLASSE'])

In [33]:
data_train.head()

,ESCT,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE
0,1,0,360,0,313,9,52,0,25,48,0,1
1,0,0,350,1,468,10,65,0,33,6,0,1
2,0,0,1100,0,829,9,125,0,56,48,0,1
3,0,0,3000,0,552,12,76,1,31,60,0,1
4,1,0,1000,0,809,12,111,0,24,7,0,1


In [34]:
data_test.head()

,ESCT,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE
0,0,2,500,1,618,10,85,0,36,6,0,0
1,1,0,813,0,552,4,119,0,43,48,119,1
2,3,0,350,0,488,12,66,0,43,0,0,1
3,1,0,1530,0,381,1,398,0,28,48,0,1
4,0,0,688,1,396,10,60,0,49,72,0,1


In [35]:
#Atributos de treino e validação
X_train = Variable(torch.Tensor(X_train).float())
X_val = Variable(torch.Tensor(X_val).float())

#Classes de treino e validação
y_train = Variable(torch.Tensor(y_train).long())
y_val = Variable(torch.Tensor(y_val).long())

#Atributos e classes de teste
X_test = Variable(torch.Tensor(X_test).float())
y_test = Variable(torch.Tensor(y_test).long())

In [36]:
X_train[0]

tensor([  0.,   0., 430.,   0., 422.,  10.,  64.,   0.,  36.,   0.,   0.])

In [37]:
class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self).__init__()
        # 11 entradas (características) e 100 neurônios na camada oculta
        self.fc1 = nn.Linear(11, 200)
        
        # Recebe 100 neurônios da camada oculta
        # Devolve 2 neurônios como camada de saída (2 classes possíveis)
        self.fc2 = nn.Linear(200, 2)
        #self.softmax = nn.Softmax(dim=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        #X = self.softmax(X)
        X = self.sigmoid(X)

        return X

In [38]:
#Criação do objeto com a classe da Rede Neural
net = Net()

#Arquitetura da Rede Neural
print(net)

Net(
  (fc1): Linear(in_features=11, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)


In [39]:
# cross entropy loss is adequate for classification problems
criterion = nn.CrossEntropyLoss()

# set the optimizer
optimizer = torch.optim.SGD(net.parameters(), lr = 0.001)

#Ajustes manuais dos parâmetros
#Iniciado treinamentos com lr = 0.001 com 100 neurônios na camada oculta

#Ajustado lr aumento de 0.001 para 0.01
#    => Aumento da precisão da classe positiva de 0.48 para 0.79

#Ajustado aumento de 100 para 200 neurônios na camada oculta
#    => Redução da precisão da classe positiva de 0.79 para 0.21 

#Ajustado aumento de 200 para 50 neurônios na camada oculta
#    => Redução generalizada nas métricas da classe positiva
#Precisão, recall e f1 na classe 1:       0.00      0.00      0.00  

NUM_EPOCHS = 1000

# loop to train the neural net
for epoch in range(NUM_EPOCHS):
    optimizer.zero_grad()
    out = net(X_train)
    loss = criterion(out, y_train)
    loss.backward()
    optimizer.step()
    
    if epoch % 200 == 0:
        print('At epoch %d the loss is %f' % (epoch, loss.data.item()))

At epoch 0 the loss is 0.692468
At epoch 200 the loss is 0.692419
At epoch 400 the loss is 0.692418
At epoch 600 the loss is 0.692417
At epoch 800 the loss is 0.692417


* Aplicação da rede nos dados de validação

In [40]:
#Realiza predições no conjunto de testes
#Cada predição é composto por um par (predição para cada classe)
predict_out = net(X_val)


#Recupera como valor predito o maior de cada par predito
_, predict_y = torch.max(predict_out, 1)

In [41]:
#Avaliação da Rede Neural nos dados de validação

#Matriz de confusão
print('Matriz de confusão')
print(confusion_matrix(y_val.data, predict_y.data))

#Relatório de avaliação da classificação
print('-' * 60, end='\n\n')
print('Relatório de avaliação da classificação')
print(classification_report(y_val.data, predict_y.data,
                            zero_division=0))

Matriz de confusão
[[636   3]
 [553   8]]
------------------------------------------------------------

Relatório de avaliação da classificação
              precision    recall  f1-score   support

           0       0.53      1.00      0.70       639
           1       0.73      0.01      0.03       561

    accuracy                           0.54      1200
   macro avg       0.63      0.50      0.36      1200
weighted avg       0.62      0.54      0.38      1200



* Aplicação do modelo nos dados de teste

In [42]:
#Realiza predições no conjunto de testes
#Cada predição é composto por um par (predição para cada classe)
predict_out = net(X_test)


#Recupera como valor predito o maior de cada par predito
_, predict_y = torch.max(predict_out, 1)

In [43]:
#Avaliação da Rede Neural nos dados de teste

#Matriz de confusão
print('Matriz de confusão')
print(confusion_matrix(y_test.data, predict_y.data))

#Relatório de avaliação da classificação
print('-' * 60, end='\n\n')
print('Relatório de avaliação da classificação')
print(classification_report(y_test.data, predict_y.data,
                            zero_division=0))

Matriz de confusão
[[305   1]
 [270   1]]
------------------------------------------------------------

Relatório de avaliação da classificação
              precision    recall  f1-score   support

           0       0.53      1.00      0.69       306
           1       0.50      0.00      0.01       271

    accuracy                           0.53       577
   macro avg       0.52      0.50      0.35       577
weighted avg       0.52      0.53      0.37       577



In [44]:
#Visualização dos valores da saída da rede neural
predict_out[:10]

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], grad_fn=<SliceBackward>)

In [45]:
#Visualização das classes preditas
predict_y[:10]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

A etapa anterior realizou uma versão simplificada da criação, treinamento e aplicação de uma rede MLP com definição arbitrária dos hiperparâmetros. A seguir utilizamos a técnica de *Grid Search* para otimização de hiperparâmetros e obtenção de melhor capacidade de generalização da aplicação da rede neural criada.

#MLP COM SINTONIZAÇÃO DE HIPERPARÂMETROS

In [46]:
!pip install skorch

In [106]:
#Biblioteca para sintonização de Hiperparâmetro
from sklearn.model_selection import GridSearchCV
#Biblioteca para adaptação sintonização de Hiperparâmetro ao Pytorch
from skorch import NeuralNetClassifier

In [136]:
class Net_(nn.Module):
    # define nn
    def __init__(self, neur=200):
        super(Net_, self).__init__()
        # 11 entradas (características) e 100 neurônios na camada oculta
        self.fc1 = nn.Linear(11, neur)
        
        # Recebe 100 neurônios da camada oculta
        # Devolve 2 neurônios como camada de saída (2 classes possíveis)
        self.fc2 = nn.Linear(neur, 2)
        #self.softmax = nn.Softmax(dim=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        #X = self.softmax(X)
        X = self.sigmoid(X)

        return X

In [137]:
netGS = NeuralNetClassifier(
    Net_,
    max_epochs=10,
    lr=0.1,
    criterion=nn.CrossEntropyLoss,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
    )

In [138]:
params = {
    'lr': [0.1, 0.01, 0.001, 0.002], #Taxa de aprendizado
    'max_epochs': [10, 20, 100], #Número de épocas
    'module__neur': [100, 150, 200] #Neurônios na camada oculta
}

In [150]:
gs = GridSearchCV(netGS,
                  params,
                  refit=True,
                  cv=3,
                  scoring='accuracy')

In [151]:
gs.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7178       0.5000        0.7241  0.0107
      2        0.7178       0.5000        0.7241  0.0075
      3        0.7178       0.5000        0.7241  0.0078
      4        0.7178       0.5000        0.7241  0.0067
      5        0.7178       0.5000        0.7241  0.0065
      6        0.7178       0.5000        0.7241  0.0065
      7        0.7178       0.5000        0.7241  0.0081
      8        0.7178       0.5000        0.7241  0.0063
      9        0.7178       0.5000        0.7241  0.0073
     10        0.7178       0.5000        0.7241  0.0096
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7035       0.4750        0.6931  0.0041
      2        0.6931       0.4750        0.6931  0.0087
      3        0.6931       0.4750        0.6931  0.0070
      4        0.6931       0.4

GridSearchCV(cv=3, error_score=nan,
             estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.Net_'>,
),
             iid='deprecated', n_jobs=None,
             param_grid={'lr': [0.1, 0.01, 0.001, 0.002],
                         'max_epochs': [10, 20, 100],
                         'module__neur': [100, 150, 200]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [154]:
print(f'Score: {gs.best_score_:.3f}\nParâmetros:\n{gs.best_params_}')

Score: 0.530
Parâmetros:
{'lr': 0.01, 'max_epochs': 20, 'module__neur': 100}


In [155]:
y_pred = gs.predict(X_val)

In [157]:
y_pred.data

In [158]:
#Avaliação da Rede Neural nos dados de teste

#Matriz de confusão
print('Matriz de confusão')
print(confusion_matrix(y_val.data, y_pred))

#Relatório de avaliação da classificação
print('-' * 60, end='\n\n')
print('Relatório de avaliação da classificação')
print(classification_report(y_val.data, y_pred,
                            zero_division=0))

Matriz de confusão
[[639   0]
 [561   0]]
------------------------------------------------------------

Relatório de avaliação da classificação
              precision    recall  f1-score   support

           0       0.53      1.00      0.69       639
           1       0.00      0.00      0.00       561

    accuracy                           0.53      1200
   macro avg       0.27      0.50      0.35      1200
weighted avg       0.28      0.53      0.37      1200



In [159]:
import pickle

In [160]:
#Salvando modelo

# saving
with open('JANIO_T3_MLP_GS.pkl', 'wb') as f:
    pickle.dump(gs, f)
"""
# loading
with open('JANIO_T3_MLP_GS.pkl', 'rb') as f:
    model = pickle.load(f)
"""

print('Modelo salvo\nPara usar, basta reativar o código acima')

Modelo salvo
Para usar, basta reativar o código acima
